In [ ]:
import datetime
import pprint

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelBehavior, ModelName

# Initialize an Agent

In [ ]:
# Load API keys from .env file
api_keys = APIKeys()

# Initialize a chat model with your API key
chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.factual().model_dump(),
)

# Initialize an agent using the chat model
agent = create_agent(model=chat_model)

# Invoke Agent

In [ ]:
# Invoke the agent with a user message
response = agent.invoke(input={"messages": [HumanMessage(content="What's the capital of the Moon?")]})

# Print the response
pprint.pprint(response)

# Stream Response

In [ ]:
# Stream response to reduce perceived latency
for token, _ in agent.stream(
    input={"messages": [HumanMessage(content="What's the capital of the Moon?")]}, stream_mode="messages"
):
    if token.content:
        print(token.content, end="", flush=True)

# System prompts

In [ ]:
system_prompt = """
You are a science fiction writer, developing a rich, scientifically grounded
universe in the solar system in the 22nd century.

You will provide detailed information about the capitals of various celestial
bodies, including their history, culture, and economy.
"""

creative_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.creative().model_dump(),
)

scifi_agent = create_agent(
    model=creative_model,
    system_prompt=system_prompt,
)

response = scifi_agent.invoke(input={"messages": [HumanMessage(content="What's the capital of the Moon?")]})

pprint.pprint(response)

# Structured Output

In [ ]:
class CapitalInfo(BaseModel):
    """Information about the capital city of a celestial body."""

    capital: str = Field(..., description="The capital city of the given celestial body.")
    founded: datetime.date = Field(..., description="The date when the capital was founded.")
    population: int = Field(..., description="The population of the capital city.")
    description: str = Field(..., description="A brief description of the capital city.")
    primary_language: str = Field(..., description="The primary language spoken in the capital city.")
    core_industries: list[str] = Field(..., description="A list of core industries in the capital city.")
    brief_history: str = Field(..., description="A brief history of the capital city.")


scifi_agent = create_agent(
    model=creative_model,
    system_prompt=system_prompt,
    response_format=CapitalInfo,
)

response = scifi_agent.invoke(input={"messages": [HumanMessage(content="What's the capital of the Moon?")]})

response["structured_response"]